#### Import necessary Libraries

In [1]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import requests # library to handle requests
import re  # library to parse text

#### Define the corresponding URL

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#### Send the GET Request and get relevant part of the results

In [37]:
results = str(requests.get(url).text)

table=results[results.find('<table'):results.find('</table')].split("\n")
table[0:12]

['<table class="wikitable sortable">',
 '<tbody><tr>',
 '<th>Postcode</th>',
 '<th>Borough</th>',
 '<th>Neighbourhood',
 '</th></tr>',
 '<tr>',
 '<td>M1A</td>',
 '<td>Not assigned</td>',
 '<td>Not assigned',
 '</td></tr>',
 '<tr>']

### Transform HTML table it into a pandas dataframe

In [32]:
# define columns name
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
pc_df = pd.DataFrame(columns=column_names)


# extracting field from table and filling Dataframe
for i in range(len(table)) :
    if re.match('^<tr>',table[i]) :
        post_code=table[i+1][4:7]
        m=re.search('<td>(<a.*">)?(.*?)(</a>)?</td>',table[i+2])
        borough=m.group(2)
        m=re.search('<td>(<a.*">)?(.*?)(</a>)?$',table[i+3])
        neighbourhood=m.group(2)
        pc_df=pc_df.append( {'PostalCode': post_code,
                             'Borough': borough,
                             'Neighborhood': neighbourhood}, 
                             ignore_index=True)

pc_df.head()        

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Clean up dataframe

In [33]:
# exclude "Not assigned"  borough
pc_df = pc_df[pc_df['Borough'] != 'Not assigned'].reset_index(drop=True)

# replacing "Not assigned" neighborood to borough
pc_df.loc[pc_df['Neighborhood'] == 'Not assigned', 'Neighborhood' ] = pc_df['Borough']

# combine rows with same postalcodes
pc_fn=pc_df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
pc_fn.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Print size of dataframe

In [34]:
pc_fn.shape

(103, 3)